In [0]:
pip install azure-storage-blob xlrd==1.2.0

Python interpreter will be restarted.
Requirement already satisfied: azure-storage-blob in /local_disk0/.ephemeral_nfs/envs/pythonEnv-b5646bbe-f042-4749-a3b4-5b8ef7f8f218/lib/python3.8/site-packages (12.8.1)
Requirement already satisfied: xlrd==1.2.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-b5646bbe-f042-4749-a3b4-5b8ef7f8f218/lib/python3.8/site-packages (1.2.0)
Requirement already satisfied: cryptography>=2.1.4 in /databricks/python3/lib/python3.8/site-packages (from azure-storage-blob) (3.1.1)
Requirement already satisfied: msrest>=0.6.18 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-b5646bbe-f042-4749-a3b4-5b8ef7f8f218/lib/python3.8/site-packages (from azure-storage-blob) (0.6.21)
Requirement already satisfied: azure-core<2.0.0,>=1.10.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-b5646bbe-f042-4749-a3b4-5b8ef7f8f218/lib/python3.8/site-packages (from azure-storage-blob) (1.17.0)
Requirement already satisfied: cffi!=1.11.3,>=1.8 in /databricks/python3/lib/python3.8/site-packages (from cryptography>=2.1.4->azure-storage-blob) (1.14.3)
Requirement already satisfied: six>=1.4.1 in /databricks/python3/lib/python3.8/site-packages (from cryptography>=2.1.4->azure-storage-blob) (1.15.0)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from msrest>=0.6.18->azure-storage-blob) (2021.5.30)
Requirement already satisfied: requests-oauthlib>=0.5.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-b5646bbe-f042-4749-a3b4-5b8ef7f8f218/lib/python3.8/site-packages (from msrest>=0.6.18->azure-storage-blob) (1.3.0)
Requirement already satisfied: isodate>=0.6.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-b5646bbe-f042-4749-a3b4-5b8ef7f8f218/lib/python3.8/site-packages (from msrest>=0.6.18->azure-storage-blob) (0.6.0)
Requirement already satisfied: requests~=2.16 in /databricks/python3/lib/python3.8/site-packages (from msrest>=0.6.18->azure-storage-blob) (2.24.0)
Requirement already satisfied: pycparser in /databricks/python3/lib/python3.8/site-packages (from cffi!=1.11.3,>=1.8->cryptography>=2.1.4->azure-storage-blob) (2.20)
Requirement already satisfied: oauthlib>=3.0.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-b5646bbe-f042-4749-a3b4-5b8ef7f8f218/lib/python3.8/site-packages (from requests-oauthlib>=0.5.0->msrest>=0.6.18->azure-storage-blob) (3.1.1)
Requirement already satisfied: chardet<4,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests~=2.16->msrest>=0.6.18->azure-storage-blob) (3.0.4)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests~=2.16->msrest>=0.6.18->azure-storage-blob) (2.10)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests~=2.16->msrest>=0.6.18->azure-storage-blob) (1.25.11)
WARNING: You are using pip version 20.2.4; however, version 21.2.3 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-b5646bbe-f042-4749-a3b4-5b8ef7f8f218/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, ArrayType, Row
from pyspark.sql.functions import split, explode, sequence, col
from azure.storage.blob import BlobServiceClient

from pathlib import Path
from itertools import chain
import pandas as pd
import xlrd
import os

In [0]:
# Read input parameters
dbutils.widgets.text("folder", "")
folder = dbutils.widgets.get("folder")

In [0]:
# Connect to blob storage
blob_account_name = "cahospitalstorage"
blob_container_name = "chargemasters"
blob_sas_token = r"sp=racwl&st=2021-08-09T17:55:40Z&se=2021-08-10T01:55:40Z&sv=2020-08-04&sr=c&sig=Q8eHbmLaLe9wujjAPXjhoqzQcsYISGO4HHHnY8P5dYE%3D"

wasbs_path = "wasbs://%s@%s.blob.core.windows.net/" % (blob_container_name, blob_account_name)
spark.conf.set("fs.azure.sas.%s.%s.blob.core.windows.net" % (blob_container_name, blob_account_name), blob_sas_token)
print('Remote blob path: ' + wasbs_path)

Remote blob path: wasbs://chargemasters@cahospitalstorage.blob.core.windows.net/

In [0]:
# Get path of excel files
connection_string = "DefaultEndpointsProtocol=https;AccountName=cahospitalstorage;AccountKey=w3EYeDH//L+J0uscsn3BYDatbFCcB7CUXK4MfOSNmwc16v3WAQ7EPcw2IpchUrelLVuowjfugmv99e0Os8J5Sw==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(blob_container_name)

files = container_client.list_blobs(name_starts_with='raw_data/' + folder)
files_rdd = spark.sparkContext.parallelize(files).map(lambda record: record['name'])
excel_files_rdd = files_rdd.filter(lambda file: '.xlsx' in file or '.xls' in file)

In [0]:
def sheet_names(file):
    blob_data = container_client.get_blob_client(file)
    blob = blob_data.download_blob().content_as_bytes()
    try:
        df = pd.read_excel(blob, None)
        return (file, list(df.keys()))
    except:
        return (None, None)

def read_files(row):
    file = row[0]
    sheet = row[1]
    match_string = 'Evaluation & Management Services (CPT Codes 99201-99499)'
  
    blob_data = container_client.get_blob_client(file)
    blob = blob_data.download_blob().content_as_bytes()
    try:
        df = pd.read_excel(blob, sheet)
        df_string = df.to_string()
        if match_string in df_string:
            return (file, sheet)
    except:
        return (None, None)

sheetnames = excel_files_rdd.map(sheet_names).toDF(['File', 'Sheetname'])
sheetnames_expand = sheetnames.withColumn('Sheetname',explode('Sheetname'))

forms = sheetnames_expand.rdd.map(read_files).filter(lambda x: x is not None)
forms_list = forms.collect()

In [0]:
# Read each target excel sheet and concatentate all rows with CPT code
schema = StructType([
    StructField('Description', StringType(), True),
    StructField('CPT', StringType(), True),
    StructField('Charge', StringType(), False)
])
df_all = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)

for form in forms_list:
  filepath = form[0]
  sheet = form[1]
  blob_data = container_client.get_blob_client(filepath)
  try:
    with open("temp.xlsx", "wb") as temp:
      data = blob_data.download_blob()
      data.readinto(temp)
    pdf = pd.read_excel("temp.xlsx", sheet_name = sheet, 
                        usecols=[0,1,2], 
                        names = ['Description', 'CPT', 'Charge'])

    df_tmp = spark.createDataFrame(pdf, schema=schema)
    df_tmp = df_tmp.filter(df_tmp.CPT.rlike("\d{5}"))
    df_all = df_all.union(df_tmp)
  except:
    print("Cannot read xlsx file: %s" % (filepath))
    pass

/databricks/spark/python/pyspark/sql/pandas/conversion.py:300: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Expected a string or bytes object, got a 'int' object
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)
/databricks/spark/python/pyspark/sql/pandas/conversion.py:300: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Expected a string or bytes object, got a 'float' object
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)

In [0]:
# Clean up df_all
## Ex. ranges => CPT Code 97161-9763 -> 97161, 97162, 97163
## Ex. splits => CPT Code 81002 or 81003
df_ranges = df_all.filter(df_all.CPT.contains("-"))
df_split  = df_all.filter(df_all.CPT.rlike("/|or"))
df_int  = df_all.filter(~df_all.CPT.rlike("-|/|or"))

# df_ranges
df_array = df_ranges.withColumn('CPT_array', split('CPT', '-').cast(ArrayType(IntegerType())))
df_array = df_array.withColumn('CPT', explode(sequence(col('CPT_array')[0], col('CPT_array')[1])))
df_array = df_array.select(['Description', 'CPT', 'Charge'])

# df_split
df_split = df_split.withColumn('CPT',explode(split('CPT',"\/|or"))) 

# combine
df_new_rows = df_array.union(df_split)
df_final = df_new_rows.union(df_int)

In [0]:
# Export df_final to blob storage
output = "clean_data_chargemasters"
df_final.write.option("header", "true").mode("overwrite").parquet(output)
for file in os.listdir("/dbfs/" + output):
  try:
    with open("/dbfs/%s/%s" % (output, file), "rb") as data:
        container_client.upload_blob(name="%s/%s/%s" % (output, folder, file), data=data, overwrite=True)
  except Exception as e:
    print(e)

In [0]:
dbutils.notebook.exit("%s/%s/%s" % (blob_container_name, output, folder))